In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.1`
import $ivy.`sh.almond::almond-spark:0.6.0`
import org.apache.log4j.{Level, Logger}

Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = NotebookSparkSession.builder().master("local[*]").getOrCreate()
import spark.implicits._
import spark.sqlContext.implicits._

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  

import $ivy.$                              

import org.apache.log4j.{Level, Logger}


import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@1e0733dc
import spark.implicits._

import spark.sqlContext.implicits._

In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window

In [3]:
val crimeCsvPath = "crimes-in-boston/crime.csv"
val offenceCodesCsvPath = "crimes-in-boston/offense_codes.csv"
val outputFolderPath = "crimes-in-boston/outputs"

crimeCsvPath: String = "crimes-in-boston/crime.csv"
offenceCodesCsvPath: String = "crimes-in-boston/offense_codes.csv"
outputFolderPath: String = "crimes-in-boston/outputs"

In [16]:
val rawCrimes = spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(crimeCsvPath)
    .cache()

val rawOffenceCodes = spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(offenceCodesCsvPath)
    .withColumn("pretty_name", split($"NAME", "\\s*-\\s*")(0))

load at cmd15.sc:5

1 / 1

load at cmd15.sc:5

8 / 8

load at cmd15.sc:12

1 / 1

load at cmd15.sc:12

2 / 2

rawCrimes: Dataset[Row] = [INCIDENT_NUMBER: string, OFFENSE_CODE: int ... 15 more fields]
rawOffenceCodes: DataFrame = [CODE: int, NAME: string ... 1 more field]

In [17]:
rawCrimes.printSchema

root
 |-- INCIDENT_NUMBER: string (nullable = true)
 |-- OFFENSE_CODE: integer (nullable = true)
 |-- OFFENSE_CODE_GROUP: string (nullable = true)
 |-- OFFENSE_DESCRIPTION: string (nullable = true)
 |-- DISTRICT: string (nullable = true)
 |-- REPORTING_AREA: string (nullable = true)
 |-- SHOOTING: string (nullable = true)
 |-- OCCURRED_ON_DATE: timestamp (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- HOUR: integer (nullable = true)
 |-- UCR_PART: string (nullable = true)
 |-- STREET: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Location: string (nullable = true)



In [18]:
val percentileFunc = expr("percentile_approx(crimes_monthly, 0.5)");

// rawCrimes.withColumn("crimes_monthly", percentileFunc.over(perMonthWindow))

val monthlyCrimes = rawCrimes
    .groupBy($"DISTRICT", $"YEAR", $"MONTH")
    .agg(count("*").as("crimes_monthly"))
    .groupBy($"DISTRICT")
    .agg(percentileFunc.alias("crimes_monthly"))

percentileFunc: Column = percentile_approx(crimes_monthly, 0.5)
monthlyCrimes: DataFrame = [DISTRICT: string, crimes_monthly: bigint]

In [19]:
val perDistrictWindow = Window.partitionBy($"DISTRICT").orderBy(desc("crime_type_count"))
val topCrimeTypeCount = 3

val topCrimeTypes = rawCrimes
    .groupBy($"DISTRICT", $"OFFENSE_CODE")
    .agg(count("OFFENSE_CODE").as("crime_type_count"))
    .withColumn("offence_rank", rank().over(perDistrictWindow))
    .where($"offence_rank" <= topCrimeTypeCount)
    .join(broadcast(rawOffenceCodes), $"OFFENSE_CODE" === rawOffenceCodes("CODE"))
    .groupBy($"DISTRICT")
    .agg(collect_set("pretty_name").alias("frequent_crime_types"))

perDistrictWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@226c4197
topCrimeTypeCount: Int = 3
topCrimeTypes: DataFrame = [DISTRICT: string, frequent_crime_types: array<string>]

In [20]:
rawCrimes
    .groupBy($"DISTRICT")
    .agg(count("*").alias("crimes_total"),
         avg("Lat").alias("lat"),
         avg("Long").alias("lng"))
    .join(topCrimeTypes, "DISTRICT")
    .join(monthlyCrimes, "DISTRICT")
    .select($"crimes_total", $"crimes_monthly", $"frequent_crime_types", $"lat", $"lng")
    .write.parquet(outputFolderPath)

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

8 / 8

run at ThreadPoolExecutor.java:1149

...

8 / 8

parquet at cmd19.sc:9

200 / 200

parquet at cmd19.sc:9

200 / 200